In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import sys
sys.path.append('../code')
import compute_indicators as ci

In [4]:
import numpy as np
import pandas as pd
import talib as ta
from tqdm.notebook import tqdm
import time

In [5]:
import alpaca_trade_api as tradeapi

key_id = None
secret_key = None
with open('../files/private/alpakey') as key_file:
    keys = key_file.readlines()
    key_id = keys[0].strip()
    secret_key = keys[1].strip()
    
api_url = "https://paper-api.alpaca.markets"

alpaca = tradeapi.REST(key_id, secret_key, api_url, api_version='v2')

In [6]:
import json

symbols = []
with open('../files/public/sp_500.json') as top_symbols:
    symbols = json.load(top_symbols)
    
symbols.append('SPY')

In [7]:
columns = json.load(open('../models/columns.json', 'rb'))

In [8]:
from keras.models import load_model

model = load_model('../models/model.h5')

Using Theano backend.


In [9]:
from pickle import load

scaler = load(open('../models/scaler.pkl', 'rb'))

In [10]:
def buy_stock(api, symbol, qty):
    try:
        api.submit_order(
            symbol=symbol,
            side='buy',
            type='market',
            qty=qty,
            time_in_force='day'
        )
    except Exception as e:
        print(f'An error occured attempting to buy {qty} shares of {symbol}', e)  

In [11]:
def sell_stock(api, symbol, qty):
    try:
        api.submit_order(
            symbol=symbol,
            side='sell',
            type='market',
            qty=qty,
            time_in_force='day'
        )
    except Exception as e:
        print(f'An error occured attempting to sell {qty} shares of {symbol}', e)

In [12]:
def predict(stock, columns, model, scaler):
    return ci.predict_today(stock, columns, model, scaler)

In [13]:
while True:
    stocks = []

    for symbol in tqdm(symbols):
        try:
            stock = alpaca.polygon.historic_agg_v2(symbol, 5, 'minute',
                                                   _from='2020-07-22', to='2020-07-24', limit=300).df
            stock['symbol'] = symbol
            stocks.append(stock)
        except Exception as e:
            print(e)
            continue
            
    stocks = pd.concat(stocks)
    stocks.reset_index(inplace=True)
    
    stocks.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'symbol']
    stocks.set_index('symbol', inplace=True)
    stocks.date = pd.to_datetime(stocks.date, utc=True)
    
    stocks_by_symbol = stocks.groupby('symbol')
    
    open_positions = {position.symbol: position for position in alpaca.list_positions()}
    
    alpaca.cancel_all_orders()

    for symbol in tqdm(symbols):
        if symbol in stocks.index: 
            stock = stocks_by_symbol.get_group(symbol)
            ci.compute(stock)
            prediction, last_close = predict(stock, columns, model, scaler)
            if (prediction >= 0.55) and (symbol not in open_positions):
                qty = 1000 // last_close
                buy_stock(alpaca, symbol, qty)
            elif (prediction < 0.55) and (symbol in open_positions):
                qty = open_positions[symbol].qty
                sell_stock(alpaca, symbol, qty)
    
    time.sleep(10 * 60)

500 Server Error: Internal Server Error for url: https://api.polygon.io/v2/aggs/ticker/COO/range/5/minute/2020-07-22/2020-07-24?unadjusted=False&limit=300&apiKey=PKHFDGG79TL869ZQDE2E
500 Server Error: Internal Server Error for url: https://api.polygon.io/v2/aggs/ticker/PWR/range/5/minute/2020-07-22/2020-07-24?unadjusted=False&limit=300&apiKey=PKHFDGG79TL869ZQDE2E
500 Server Error: Internal Server Error for url: https://api.polygon.io/v2/aggs/ticker/VRSN/range/5/minute/2020-07-22/2020-07-24?unadjusted=False&limit=300&apiKey=PKHFDGG79TL869ZQDE2E



An error occured attempting to buy 0.0 shares of AMZN qty must be > 0



500 Server Error: Internal Server Error for url: https://api.polygon.io/v2/aggs/ticker/ADP/range/5/minute/2020-07-22/2020-07-24?unadjusted=False&limit=300&apiKey=PKHFDGG79TL869ZQDE2E
500 Server Error: Internal Server Error for url: https://api.polygon.io/v2/aggs/ticker/AIV/range/5/minute/2020-07-22/2020-07-24?unadjusted=False&limit=300&apiKey=PKHFDGG79TL869ZQDE2E
500 Server Error: Internal Server Error for url: https://api.polygon.io/v2/aggs/ticker/AIZ/range/5/minute/2020-07-22/2020-07-24?unadjusted=False&limit=300&apiKey=PKHFDGG79TL869ZQDE2E
500 Server Error: Internal Server Error for url: https://api.polygon.io/v2/aggs/ticker/ATO/range/5/minute/2020-07-22/2020-07-24?unadjusted=False&limit=300&apiKey=PKHFDGG79TL869ZQDE2E
500 Server Error: Internal Server Error for url: https://api.polygon.io/v2/aggs/ticker/CNC/range/5/minute/2020-07-22/2020-07-24?unadjusted=False&limit=300&apiKey=PKHFDGG79TL869ZQDE2E
500 Server Error: Internal Server Error for url: https://api.polygon.io/v2/aggs/ticke

An error occured attempting to buy 0.0 shares of AMZN qty must be > 0



500 Server Error: Internal Server Error for url: https://api.polygon.io/v2/aggs/ticker/BAC/range/5/minute/2020-07-22/2020-07-24?unadjusted=False&limit=300&apiKey=PKHFDGG79TL869ZQDE2E
500 Server Error: Internal Server Error for url: https://api.polygon.io/v2/aggs/ticker/BMY/range/5/minute/2020-07-22/2020-07-24?unadjusted=False&limit=300&apiKey=PKHFDGG79TL869ZQDE2E
500 Server Error: Internal Server Error for url: https://api.polygon.io/v2/aggs/ticker/FITB/range/5/minute/2020-07-22/2020-07-24?unadjusted=False&limit=300&apiKey=PKHFDGG79TL869ZQDE2E
500 Server Error: Internal Server Error for url: https://api.polygon.io/v2/aggs/ticker/PPL/range/5/minute/2020-07-22/2020-07-24?unadjusted=False&limit=300&apiKey=PKHFDGG79TL869ZQDE2E
500 Server Error: Internal Server Error for url: https://api.polygon.io/v2/aggs/ticker/SNA/range/5/minute/2020-07-22/2020-07-24?unadjusted=False&limit=300&apiKey=PKHFDGG79TL869ZQDE2E



...

500 Server Error: Internal Server Error for url: https://api.polygon.io/v2/aggs/ticker/WYNN/range/5/minute/2020-07-22/2020-07-24?unadjusted=False&limit=300&apiKey=PKHFDGG79TL869ZQDE2E
500 Server Error: Internal Server Error for url: https://api.polygon.io/v2/aggs/ticker/ZBH/range/5/minute/2020-07-22/2020-07-24?unadjusted=False&limit=300&apiKey=PKHFDGG79TL869ZQDE2E
500 Server Error: Internal Server Error for url: https://api.polygon.io/v2/aggs/ticker/SPY/range/5/minute/2020-07-22/2020-07-24?unadjusted=False&limit=300&apiKey=PKHFDGG79TL869ZQDE2E



An error occured attempting to buy 0.0 shares of AMZN qty must be > 0



An error occured attempting to buy 0.0 shares of AMZN qty must be > 0



An error occured attempting to buy 0.0 shares of AMZN qty must be > 0



KeyboardInterrupt: 